<a href="https://colab.research.google.com/github/SylviaNice/7162856-G-rez-Git-et-GitHub/blob/main/Projet_Bottleneck_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Python - Optimiser la gestion des données pour Bottleneck

In [1]:
# 1. Nettoyage de l'environnement (efface toutes les variables en mémoire)

%reset -f

In [2]:
# 2. Installation des bibliothèques

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 3. Importation des données

from google.colab import files
uploaded = files.upload()

Saving Fichier_web.xlsx to Fichier_web.xlsx
Saving Fichier_erp.xlsx to Fichier_erp.xlsx
Saving fichier_liaison.xlsx to fichier_liaison.xlsx


In [4]:
# 4. Chargement des 3 fichiers

erp_file = "Fichier_erp.xlsx"
web_file = "Fichier_web.xlsx"
liaison_file = "fichier_liaison.xlsx"

In [5]:
# 5. Lecture des fichiers

erp_df = pd.read_excel(erp_file)
web_df = pd.read_excel(web_file)
liaison_df = pd.read_excel(liaison_file)

In [6]:
# 6. Aperçu des premières lignes des fichiers importés

display(erp_df.head())
display(web_df.head())
display(liaison_df.head())

,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,16004,0,0,0,0.0,5.0,NaN,NaN,2.0,2018-06-07 16:27:25,2018-06-07 14:27:25,NaN,Château du Couvent Pomerol 2017,NaN,publish,closed,closed,NaN,chateau-du-couvent-pomerol-2017,2020-08-25 18:35:02,2020-08-25 16:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15075,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-14 15:39:43,2018-02-14 14:39:43,NaN,Parés Baltà Penedès Indigena 2017,Des couleurs et aromes intenses où le fruit et...,publish,closed,closed,NaN,pares-balta-penedes-indigena-2017,2020-08-20 15:35:02,2020-08-20 13:35:02,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,16209,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-14 17:15:31,2018-02-14 16:15:31,NaN,Maurel Cabardès Tradition 2017,"Un joli nez aux arômes de fruits rouges, de ca...",publish,closed,closed,NaN,maurel-cabardes-tradition-2017,2020-08-05 18:05:03,2020-08-05 16:05:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,15763,0,0,0,0.0,1.0,NaN,NaN,2.0,2020-01-25 14:08:16,2020-01-25 13:08:16,NaN,Domaine de la Monardière Vacqueyras Vieilles V...,NaN,publish,closed,closed,NaN,domaine-de-la-monardiere-vacqueyras-les-vieill...,2020-08-21 11:35:02,2020-08-21 09:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815


In [7]:
# 7. Nombre de lignes et de colonnes de chacun des 3 fichiers

print(erp_df.shape)
print(web_df.shape)
print(liaison_df.shape)

(825, 5)
(1513, 28)
(825, 2)


In [8]:
# 8. Affichage du nom et du type de chaque colonne pour les fichiers erp, web et liaison

print(erp_df.dtypes)
print("-" * 40)
print(web_df.dtypes)
print("-" * 40)
print(liaison_df.dtypes)
print("-" * 40)

product_id          int64
onsale_web          int64
price             float64
stock_quantity      int64
stock_status       object
dtype: object
----------------------------------------
sku                              object
virtual                           int64
downloadable                      int64
rating_count                      int64
average_rating                  float64
total_sales                     float64
tax_status                       object
tax_class                       float64
post_author                     float64
post_date                datetime64[ns]
post_date_gmt            datetime64[ns]
post_content                    float64
post_title                       object
post_excerpt                     object
post_status                      object
comment_status                   object
ping_status                      object
post_password                   float64
post_name                        object
post_modified            datetime64[ns]
post_modified_g

In [9]:
# 9. Supprimer les colonnes inutiles

columns_to_drop_exist = [
    "virtual",
    "downloadable",
    "rating_count",
    "average_rating",
    "tax_class",
    "post_content",
    "post_status",
    "comment_status",
    "ping_status",
    "post_password",
    "post_content_filtered",
    "post_parent",
    "menu_order",
    "comment_count"
]
columns_to_drop_exist = [col for col in columns_to_drop_exist if col in web_df.columns]
web_df = web_df.drop(columns=columns_to_drop_exist)
print(web_df.dtypes)

sku                          object
total_sales                 float64
tax_status                   object
post_author                 float64
post_date            datetime64[ns]
post_date_gmt        datetime64[ns]
post_title                   object
post_excerpt                 object
post_name                    object
post_modified        datetime64[ns]
post_modified_gmt    datetime64[ns]
guid                         object
post_type                    object
post_mime_type               object
dtype: object


In [10]:
# 10. Renommer la colonne (rectification de l'erreur de la stagiaire)

liaison_df = pd.read_excel("fichier_liaison.xlsx")
liaison_df = liaison_df.rename(columns={'id_web': 'sku'})
print(liaison_df)

     product_id      sku
0          3847    15298
1          3849    15296
2          3850    15300
3          4032    19814
4          4039    19815
..          ...      ...
820        7203      NaN
821        7204      NaN
822        7247  13127-1
823        7329  14680-1
824        7338    16230

[825 rows x 2 columns]


In [12]:
# 11. Affichage des données manquantes

print(liaison_df[liaison_df.isna().any(axis=1)])

     product_id  sku
19         4055  NaN
49         4090  NaN
50         4092  NaN
119        4195  NaN
131        4209  NaN
..          ...  ...
817        7196  NaN
818        7200  NaN
819        7201  NaN
820        7203  NaN
821        7204  NaN

[91 rows x 2 columns]


In [13]:
# 12. Détecter les colonnes avec des valeurs manquantes pour les fichiers

print("Colonnes avec valeurs manquantes - ERP")
print(erp_df.isna().sum()
[erp_df.isna().sum() > 0])
print("-" * 40)

print("Colonnes avec valeurs manquantes - Web")
print(web_df.isna().sum()
[web_df.isna().sum() > 0])
print("-" * 40)

print("Colonnes avec valeurs manquantes - Liaison")
print(liaison_df.isna().sum()
[liaison_df.isna().sum() > 0])
print("-" * 40)

Colonnes avec valeurs manquantes - ERP
Series([], dtype: int64)
----------------------------------------
Colonnes avec valeurs manquantes - Web
sku                   85
total_sales           83
tax_status           797
post_author           83
post_date             83
post_date_gmt         83
post_title            83
post_excerpt         797
post_name             83
post_modified         83
post_modified_gmt     83
guid                  83
post_type             83
post_mime_type       799
dtype: int64
----------------------------------------
Colonnes avec valeurs manquantes - Liaison
sku    91
dtype: int64
----------------------------------------


In [14]:
# 13. Détecter le nombre de doublons pour chaque fichier

print(erp_df.duplicated().sum())
print(web_df.duplicated().sum())
print(liaison_df.duplicated().sum())

0
82
0


In [15]:
# 14. Visualiser les doublons

doublons_erp = erp_df.duplicated(keep=False)
doublons_web = web_df.duplicated(keep=False)
doublons_liaison = liaison_df.duplicated(keep=False)
print(doublons_erp, doublons_web, doublons_liaison)

0      False
1      False
2      False
3      False
4      False
       ...  
820    False
821    False
822    False
823    False
824    False
Length: 825, dtype: bool 0       False
1        True
2       False
3       False
4       False
        ...  
1508    False
1509    False
1510    False
1511    False
1512    False
Length: 1513, dtype: bool 0      False
1      False
2      False
3      False
4      False
       ...  
820    False
821    False
822    False
823    False
824    False
Length: 825, dtype: bool


In [16]:
# 15. Supprimer les doublons

erp_df.drop_duplicates(inplace=True)
web_df.drop_duplicates(inplace=True)
liaison_df.drop_duplicates(inplace=True)
print(len(erp_df), len(web_df), len(liaison_df))

825 1431 825


In [17]:
# 16. Vérifier que les doublons ont bien été supprimés

print("Doublons ERP :", erp_df.duplicated().sum())
print("Doublons Web :", web_df.duplicated().sum())
print("Doublons Liaison :", liaison_df.duplicated().sum())
print(erp_df.shape)
print(web_df.shape)
print(liaison_df.shape)

Doublons ERP : 0
Doublons Web : 0
Doublons Liaison : 0
(825, 5)
(1431, 14)
(825, 2)


In [18]:
# 17. Supprimer les lignes entièrement vides de mes fichiers

web_df.dropna(how='all', inplace=True)
erp_df.dropna(how='all', inplace=True)
liaison_df.dropna(how='all', inplace=True)
print(len(erp_df), len(web_df), len(liaison_df))

825 1430 825


In [19]:
# 18. Enregistrement des fichiers nettoyés erp et liaison

erp_df.to_excel("Fichier_erp_clean.xlsx", index=False)
liaison_df.to_excel("fichier_liaison_clean.xlsx", index=False)
web_df.to_excel("Fichier_web_clean.xlsx", index=False)
files.download("Fichier_web_clean.xlsx")
files.download("Fichier_erp_clean.xlsx")
files.download("fichier_liaison_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# 19. Supprimer les doublons sur les clés avant la fusion

erp_df = erp_df.drop_duplicates(subset=['product_id'])
liaison_df = liaison_df.drop_duplicates(subset=['product_id', 'sku'])
web_df = web_df.drop_duplicates(subset=['sku'])

In [21]:
# 20. Fusion du fichier erp avec le fichier liaison

erp_df = pd.read_excel("Fichier_erp_clean.xlsx")
liaison_df = pd.read_excel("fichier_liaison_clean.xlsx")
erp_liaison = pd.merge(erp_df, liaison_df, on='product_id', how='inner')
print(erp_liaison)
print(erp_liaison.shape)

     product_id  onsale_web  price  stock_quantity stock_status      sku
0          3847           1   24.2               0   outofstock    15298
1          3849           1   34.3               0   outofstock    15296
2          3850           1   20.8               0   outofstock    15300
3          4032           1   14.1               0   outofstock    19814
4          4039           1   46.0               0   outofstock    19815
..          ...         ...    ...             ...          ...      ...
820        7203           0   45.0              30      instock      NaN
821        7204           0   45.0               9      instock      NaN
822        7247           1   54.8              23      instock  13127-1
823        7329           0   26.5              14      instock  14680-1
824        7338           1   16.3              45      instock    16230

[825 rows x 6 columns]
(825, 6)


In [22]:
# 21. Fusion du fichier erp_liaison et du fichier web

erp_liaison_web = pd.merge(erp_liaison, web_df, on='sku', how='left')
print(erp_liaison_web.shape)
print(erp_liaison_web.dtypes)

(825, 19)
product_id                    int64
onsale_web                    int64
price                       float64
stock_quantity                int64
stock_status                 object
sku                          object
total_sales                 float64
tax_status                   object
post_author                 float64
post_date            datetime64[ns]
post_date_gmt        datetime64[ns]
post_title                   object
post_excerpt                 object
post_name                    object
post_modified        datetime64[ns]
post_modified_gmt    datetime64[ns]
guid                         object
post_type                    object
post_mime_type               object
dtype: object


In [23]:
# 22. Enregistrement et téléchargement du nouveau fichier de travail

erp_liaison_web.to_excel("erp_liaison_web.xlsx", index=False)
from google.colab import files
files.download("erp_liaison_web.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# 23. Visualiser la cellule erreur "bon-cadeau-25-euros"

print(erp_liaison_web.loc[440:449, ['sku']])

                     sku
440                13379
441                15338
442                15337
443  bon-cadeau-25-euros
444                15737
445                15958
446                16515
447                16586
448                11225
449                  NaN


In [25]:
# 24. Remplacer la cellule "bon-cadeau-25-euros" par une cellule vide

erp_liaison_web.loc[443, 'sku'] = "NaN"
print(erp_liaison_web.loc[440:449, ['sku']])

       sku
440  13379
441  15338
442  15337
443    NaN
444  15737
445  15958
446  16515
447  16586
448  11225
449    NaN


In [26]:
# 25. Détecter les incohérences de prix : prix négatifs

erp_df[erp_df['price'] <= 0]

,product_id,onsale_web,price,stock_quantity,stock_status


In [33]:
# 26. Tranformer les valeurs négatives des prix en valeurs positives

erp_df['price'] = erp_df['price'].abs()
erp_df[erp_df['price'] <= 0]

,product_id,onsale_web,price,stock_quantity,stock_status


In [37]:
# 27. Calcul du CA par ligne et par colonne

erp_liaison_web['CA'] = erp_liaison_web['price'] * erp_liaison_web['total_sales']
CA_total = erp_liaison_web['CA'].sum()
print(CA_total)
print(erp_liaison_web.columns)
print(erp_liaison_web[['price', 'total_sales', 'CA']].head())

74915.4
Index(['product_id', 'onsale_web', 'price', 'stock_quantity', 'stock_status',
       'sku', 'total_sales', 'tax_status', 'post_author', 'post_date',
       'post_date_gmt', 'post_title', 'post_excerpt', 'post_name',
       'post_modified', 'post_modified_gmt', 'guid', 'post_type',
       'post_mime_type', 'CA'],
      dtype='object')
   price  total_sales     CA
0   24.2          6.0  145.2
1   34.3          0.0    0.0
2   20.8          0.0    0.0
3   14.1          3.0   42.3
4   46.0          0.0    0.0
